In [ ]:
## Module import
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cartopy.crs as ccrs
import matplotlib.ticker as mticker
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from scipy import stats
import seaborn as sns
import cartopy.feature as cft
import matplotlib.path as mpath
import cmocean as cm
import string
import math
from datetime import datetime, timedelta
import proplot as pplt
import xesmf as xe
import netCDF4 as nc

## CMIP6 output

In [ ]:
# open IPSL file
netCDF_TA95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/data_IPSL_tas_for_std_19952014.nc'
TA_CMIP95 = xr.open_mfdataset(netCDF_TA95_file)

netCDF_PR95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/data_IPSL_pr_for_std_19952014.nc'
PR_CMIP95 = xr.open_mfdataset(netCDF_PR95_file)

netCDF_PS95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/data_IPSL_psl2_for_std_19952014.nc'
PS_CMIP95 = xr.open_mfdataset(netCDF_PS95_file)

netCDF_SMB95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/IPSL/data_IPSL_smb_for_std_19952014.nc'
SMB_CMIP95 = xr.open_mfdataset(netCDF_SMB95_file)

# open MPI file
netCDF_MPI_TA95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/data_MPI_tas_for_std_19952014.nc'
TA_MPI_CMIP95 = xr.open_mfdataset(netCDF_MPI_TA95_file)

netCDF_MPI_PR95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/data_MPI_pr_for_std_19952014.nc'
PR_MPI_CMIP95 = xr.open_mfdataset(netCDF_MPI_PR95_file)

netCDF_MPI_PS95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/data_MPI_psl2_for_std_19952014.nc'
PS_MPI_CMIP95 = xr.open_mfdataset(netCDF_MPI_PS95_file)

netCDF_MPI_SMB95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/data_MPI_smb_for_std_19952014.nc'
SMB_MPI_CMIP95 = xr.open_mfdataset(netCDF_MPI_SMB95_file)

# open UKESM file
netCDF_UKESM_TA95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/data_UKESM_tas_for_std_19952014.nc'
TA_UKESM_CMIP95 = xr.open_mfdataset(netCDF_UKESM_TA95_file)

netCDF_UKESM_PR95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/data_UKESM_pr_for_std_19952014.nc'
PR_UKESM_CMIP95 = xr.open_mfdataset(netCDF_UKESM_PR95_file)

netCDF_UKESM_PS95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/data_UKESM_psl2_for_std_19952014.nc'
PS_UKESM_CMIP95 = xr.open_mfdataset(netCDF_UKESM_PS95_file)

netCDF_UKESM_SMB95_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/data_UKESM_smb_for_std_19952014.nc'
SMB_UKESM_CMIP95 = xr.open_mfdataset(netCDF_UKESM_SMB95_file)

# open topo
netCDF_topom_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/MPI/deptho_Ofx_MPI-ESM1-2-HR_piControl_r1i1p1f1_gn.nc'
TOPOM = xr.open_mfdataset(netCDF_topom_file)

netCDF_topou_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/UKESM/deptho_Ofx_UKESM1-0-LL_piControl_r1i1p1f2_gn.nc'
TOPOU = xr.open_mfdataset(netCDF_topou_file)


# Mask
netCDF_ess_file = '/Users/jcaillet/Documents/Elmer/CMIP6/DATA/RCM_ice_regrid_04000m.nc2'
MASK = xr.open_mfdataset(netCDF_ess_file)

SMB_UKESM_CMIP95['maskAIS'] = SMB_UKESM_CMIP95['lon'] - SMB_UKESM_CMIP95['lon'] + MASK['AIS'].values
SMB_UKESM_CMIP95['maskGRD'] = SMB_UKESM_CMIP95['lon'] - SMB_UKESM_CMIP95['lon'] + MASK['GROUND'].values

## OBSERVATIONS

In [ ]:
# Tair in Kelvin
# Precipitation in m_wat.eq/jour
# Pressure en Pa
netCDF_file0 = f'/Users/jcaillet/Documents/Elmer/CMIP6/INTERNAL_VARIABILITY_MULTIMODEL/ERA5_DATA.nc'
OBS = xr.open_mfdataset(netCDF_file0)
OBS = OBS.mean('time')
OBS = OBS.rename({'longitude':'lon','latitude':'lat'})
OBS['tp']=OBS['tp']*1000

## FIGURES

In [ ]:
sns.set_context('paper')

land_50m = cft.NaturalEarthFeature('physical', 'land', '50m', edgecolor='salmon', facecolor='papayawhip', linewidth=0.5)
theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
levels = np.arange(980,1045,5)

fig, axs = plt.subplots(nrows = 3, ncols = 4, subplot_kw={'projection': ccrs.SouthPolarStereo()}, figsize = (11.5, 8), constrained_layout=True)
axs = axs.flatten()


for n, ax in enumerate(axs):
    ax.text(0.025, 0.93, string.ascii_lowercase[n]+')', transform=ax.transAxes, color='black', fontweight="bold")
    #ax.text(0.0, 1.0, label, transform=ax.transAxes + trans, fontsize='medium', verticalalignment='top', fontfamily='serif')
    ax.set_extent([-180, 180, -90, -63], crs=ccrs.PlateCarree())
    #ax.add_feature(land_50m, color=[0.8, 0.8, 0.8])
    ax.coastlines(resolution='50m', color='black',)
    gl0 = ax.gridlines(draw_labels=False, linewidth=0.75, color='gray', alpha=0.7, linestyle='--', dms=True, x_inline=False, y_inline=False)
    gl0.top_labels = False
    gl0.right_labels = False
    gl0.xlocator = mticker.FixedLocator([-180, -135, -90, -45, 0, 45, 90, 135, 180])
    gl0.ylocator = mticker.FixedLocator([-90, -80, -70, -60])
    gl0.xformatter = LongitudeFormatter()
    gl0.yformatter = LatitudeFormatter()
    gl0.xlabel_style = {'size': 12, 'color': 'gray'}
    gl0.ylabel_style = {'size': 12, 'color': 'gray'}
    #ax.text(0, 0.9, '('+string.ascii_lowercase[n]+')', transform=ax.transAxes, size=14)   
    #ax.set_boundary(circle, transform=ax.transAxes)
    
fmt = {}
strs = ['980', '985','990', '995', '1000', '1005', '1010', '1015', '1020', '1025', '1030', '1035', '1040']
al = np.array([ 980., 985., 990., 995., 1000., 1005., 1010., 1015., 1020., 1025., 1030., 1035.,1040.])
for l, s in zip(al, strs):
    fmt[l] = s
    
plt.figtext(0.01, 0.8, '1995-2014 multi-member mean', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.025, 0.8, 'of Surface Mass Balance', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.01, 0.485, '1995-2014 multi-member mean', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.025, 0.485, 'of air temperature', verticalalignment='center', rotation=90, fontweight="bold")
plt.figtext(0.422, 0.97, 'MPI', fontweight="bold")
plt.figtext(0.646, 0.97, 'IPSL', fontweight="bold")
plt.figtext(0.86, 0.97, 'UKESM', fontweight="bold")
plt.figtext(0.165, 0.97, 'ERA5 Reanalysis', fontweight="bold")
plt.figtext(0.01, 0.165, '1995-2014 multi-member mean', horizontalalignment='left', verticalalignment='center', rotation='vertical', fontweight="bold")
plt.figtext(0.025, 0.165, 'of precipitation', horizontalalignment='left', verticalalignment='center', rotation='vertical', fontweight="bold")

# smb 1995-2014
p00 = axs[0].remove()
p6 = axs[2].pcolormesh(SMB_CMIP95.lon, SMB_CMIP95.lat, SMB_CMIP95['smb_mean'].where(SMB_UKESM_CMIP95['maskAIS']>0)*31536000,  cmap=cm.cm.haline, vmin = 0, vmax =1400, transform=ccrs.PlateCarree(), shading='auto')
p7 = axs[1].pcolormesh(SMB_MPI_CMIP95.lon, SMB_MPI_CMIP95.lat, SMB_MPI_CMIP95['smb_mean'].where(SMB_UKESM_CMIP95['maskAIS']>0)*31536000,  cmap=cm.cm.haline, vmin = 0, vmax =1400,transform=ccrs.PlateCarree(), shading='auto')
p8 = axs[3].pcolormesh(SMB_UKESM_CMIP95.lon, SMB_UKESM_CMIP95.lat, SMB_UKESM_CMIP95['smb_mean'].where(SMB_UKESM_CMIP95['maskAIS']>0)*31536000,  cmap=cm.cm.haline, vmin = 0, vmax =1400,transform=ccrs.PlateCarree(), shading='auto')

# Air temperature 1995-2014
p01 = axs[4].pcolormesh(OBS.lon, OBS.lat, OBS['t2m']-273.15, cmap=cm.cm.thermal, vmin = -50, vmax =10, transform=ccrs.PlateCarree(), shading='auto')
p0 = axs[6].pcolormesh(TA_CMIP95.lon, TA_CMIP95.lat, TA_CMIP95['tas_mean']-273.15, cmap=cm.cm.thermal, vmin = -50, vmax =10, transform=ccrs.PlateCarree(), shading='auto')
p1 = axs[5].pcolormesh(TA_MPI_CMIP95.lon, TA_MPI_CMIP95.lat, TA_MPI_CMIP95['tas_mean']-273.15, cmap=cm.cm.thermal, vmin = -50, vmax =10, transform=ccrs.PlateCarree(), shading='auto')
p2 = axs[7].pcolormesh(TA_UKESM_CMIP95.lon, TA_UKESM_CMIP95.lat, TA_UKESM_CMIP95['tas_mean']-273.15, cmap=cm.cm.thermal, vmin = -50, vmax =10, transform=ccrs.PlateCarree(), shading='auto')

# precipitation 1995-2014 + contour pressure
p02 = axs[8].pcolormesh(OBS.lon, OBS.lat, OBS['tp']*365, vmin = 0, vmax =2000, cmap=cm.cm.rain,  transform=ccrs.PlateCarree(), shading='auto')
p022 = axs[8].contour(OBS.lon, OBS.lat, OBS['msl']*1e-2, levels, colors='crimson', linewidths = 1, transform=ccrs.PlateCarree())
axs[8].clabel(p022, p022.levels, inline=True, fmt=fmt, fontsize=5)


p3 = axs[9].pcolormesh(PR_CMIP95.lon, PR_CMIP95.lat, PR_CMIP95['pr_mean']*31536000, vmin = 0, vmax =2000, cmap=cm.cm.rain,  transform=ccrs.PlateCarree(), shading='auto')
p33 = axs[9].contour(PS_CMIP95.lon, PS_CMIP95.lat, PS_CMIP95['psl_mean']*1e-2, levels, colors='crimson', linewidths = 1, transform=ccrs.PlateCarree())
axs[9].clabel(p33, p33.levels, inline=True, fmt=fmt, fontsize=5)

p4 = axs[10].pcolormesh(PR_MPI_CMIP95.lon, PR_MPI_CMIP95.lat, PR_MPI_CMIP95['pr_mean']*31536000, cmap=cm.cm.rain, vmin = 0, vmax =2000, transform=ccrs.PlateCarree(), shading='auto')
p44 = axs[10].contour(PS_MPI_CMIP95.lon, PS_MPI_CMIP95.lat, PS_MPI_CMIP95['psl_mean']*1e-2, levels, colors='crimson', linewidths = 1, transform=ccrs.PlateCarree())
axs[10].clabel(p44, p44.levels, inline=True, fmt=fmt, fontsize=5)

p5 = axs[11].pcolormesh(PR_UKESM_CMIP95.lon, PR_UKESM_CMIP95.lat, PR_UKESM_CMIP95['pr_mean']*31536000, cmap=cm.cm.rain, vmin = 0, vmax =2000, transform=ccrs.PlateCarree(), shading='auto')
p55 = axs[11].contour(PS_UKESM_CMIP95.lon, PS_UKESM_CMIP95.lat, PS_UKESM_CMIP95['psl_mean']*1e-2, levels, colors='crimson', linewidths = 1, transform=ccrs.PlateCarree())
axs[11].clabel(p55, p55.levels, fmt=fmt, fontsize=5)


ax_cb = plt.axes([0.05, 0.705, 0.007, 0.2 ])
cb = plt.colorbar(p8, cax=ax_cb, orientation='vertical')
cb.ax.set_ylabel('Mean SMB (kg m$^{2}$ yr$^{-1}$)');

ax_cb1 = plt.axes([0.05, 0.385, 0.007, 0.2 ])
cb1 = plt.colorbar(p2, cax=ax_cb1, orientation='vertical')
cb1.ax.set_ylabel('Mean temperature (°C)');

ax_cb2 = plt.axes([0.05, 0.07, 0.007, 0.2 ])
cb2 = plt.colorbar(p5, cax=ax_cb2, orientation='vertical')
cb2.ax.set_ylabel('Mean precipitation (kg m$^{2}$ yr$^{-1}$)');
        
plt.tight_layout(rect=[0.10, 0.0, 1, 0.97])
plt.savefig('/Users/jcaillet/Documents/Elmer/CMIP6/FIGURES/atmo_mean2.png',dpi=200)
